# Logistic Regression (03/11/2025)

In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

data = {
    'monthly_charges': np.random.normal(50, 20, 1000),
    'contract_duration_months': np.random.randint(1, 48, 1000),
    'churn': np.random.binomial(1, 0.4, 1000) # Baseline churn probability
}
df = pd.DataFrame(data)

# Introduce a relationship to make the model meaningful
df['churn'] = np.where(df['monthly_charges'] > 60, np.random.binomial(1, 0.6), df['churn'])
df['churn'] = np.where(df['contract_duration_months'] > 24, np.random.binomial(1, 0.2), df['churn'])

# Display the first few rows
print(df.head())

# Define dependent and independent variables
X = df[['monthly_charges', 'contract_duration_months']]
y = df['churn']

   monthly_charges  contract_duration_months  churn
0        52.435729                        29      0
1        44.527103                        22      1
2        44.906903                        43      0
3        39.200532                        12      1
4        63.604767                        23      0


In [2]:
# Add a constant for the intercept term
X_const = sm.add_constant(X)

# Fit the logistic regression model
logit_model = sm.Logit(y, X_const)
logit_result = logit_model.fit()

# Print the model summary
print(logit_result.summary())

Optimization terminated successfully.
         Current function value: 0.296495
         Iterations 7
                           Logit Regression Results                           
Dep. Variable:                  churn   No. Observations:                 1000
Model:                          Logit   Df Residuals:                      997
Method:                           MLE   Df Model:                            2
Date:                Mon, 03 Nov 2025   Pseudo R-squ.:                  0.2288
Time:                        11:41:08   Log-Likelihood:                -296.49
converged:                       True   LL-Null:                       -384.48
Covariance Type:            nonrobust   LLR p-value:                 6.138e-39
                               coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------
const                        1.9213      0.359      5.348      0.000       1.217

In [3]:
# Get the coefficients and confidence intervals from the results
params = logit_result.params
conf = logit_result.conf_int()
conf['Odds Ratio'] = params
conf.columns = ['Lower CI', 'Upper CI', 'Log-Odds Coeff']

# Calculate the Odds Ratio and its 95% confidence interval
odds_ratios = np.exp(conf)
print("\nOdds Ratios and 95% Confidence Intervals:\n", odds_ratios)

# Interpretation
# - Odds Ratio > 1: Increases the odds of the outcome.
# - Odds Ratio < 1: Decreases the odds of the outcome.
# - Odds Ratio = 1: No effect on the odds.


Odds Ratios and 95% Confidence Intervals:
                           Lower CI   Upper CI  Log-Odds Coeff
const                     3.377624  13.811250        6.830022
monthly_charges           0.948224   0.971606        0.959844
contract_duration_months  0.883205   0.919900        0.901366


In [4]:
def hosmer_lemeshow_test(y_true, y_prob, bins=10):
    """
    Performs the Hosmer-Lemeshow test.
    """
    df = pd.DataFrame({'y_true': y_true, 'y_prob': y_prob})
    df['decile'] = pd.qcut(df['y_prob'], bins, labels=False, duplicates='drop')

    observed_events = df.groupby('decile')['y_true'].sum()
    observed_non_events = df.groupby('decile')['y_true'].count() - observed_events
    
    expected_events = df.groupby('decile')['y_prob'].sum()
    expected_non_events = df.groupby('decile')['y_prob'].count() - expected_events
    
    hl_statistic = np.sum(
        (observed_events - expected_events)**2 / expected_events +
        (observed_non_events - expected_non_events)**2 / expected_non_events
    )
    
    # Degrees of freedom is (number of bins - 2)
    degrees_of_freedom = len(observed_events) - 2
    
    # Get p-value from chi-squared distribution
    from scipy.stats import chi2
    p_value = 1 - chi2.cdf(hl_statistic, degrees_of_freedom)

    return hl_statistic, p_value

# Get predicted probabilities
y_prob = logit_result.predict(X_const)

# Perform the Hosmer-Lemeshow test
hl_statistic, hl_p_value = hosmer_lemeshow_test(y, y_prob)

print(f"\nHosmer-Lemeshow Test:")
print(f"  - Statistic: {hl_statistic:.4f}")
print(f"  - P-value: {hl_p_value:.4f}")
if hl_p_value > 0.05:
    print("  - Conclusion: Model is a good fit (Fail to reject H0).")
else:
    print("  - Conclusion: Model is a poor fit (Reject H0).")


Hosmer-Lemeshow Test:
  - Statistic: 19.0531
  - P-value: 0.0146
  - Conclusion: Model is a poor fit (Reject H0).


In [5]:
from sklearn.metrics import roc_auc_score

# Calculate the concordance (C-statistic)
c_statistic = roc_auc_score(y, y_prob)
print(f"\nConcordance (C-statistic/AUC): {c_statistic:.4f}")

# Interpretation of C-statistic
# - 0.5: No better than random chance.
# - >0.7: Considered a good model.
# - >0.8: Considered a strong model.
# - 1.0: Perfect discrimination.


Concordance (C-statistic/AUC): 0.8398


In [6]:
# Combine actuals and predicted probabilities
analysis_df = pd.DataFrame({'actuals': y, 'predicted_prob': y_prob})

# Create decile groups based on predicted probabilities
analysis_df['decile'] = pd.qcut(analysis_df['predicted_prob'], 10, labels=False, duplicates='drop')
analysis_df = analysis_df.sort_values('decile', ascending=False)

# Calculate the event rate per decile
rank_ordering_df = analysis_df.groupby('decile').agg(
    total_customers=('actuals', 'count'),
    events=('actuals', 'sum')
)
rank_ordering_df['event_rate'] = rank_ordering_df['events'] / rank_ordering_df['total_customers']
rank_ordering_df.index.name = 'Decile (1=Highest Prob)'
print("\nRank Ordering (Decile Analysis):")
print(rank_ordering_df)

# Interpretation
# - The `event_rate` should generally decrease from Decile 1 (highest probability) to Decile 10 (lowest probability).
# - A consistent decrease indicates good rank ordering, confirming the model effectively segments customers based on their likelihood of churning.


Rank Ordering (Decile Analysis):
                         total_customers  events  event_rate
Decile (1=Highest Prob)                                     
0                                    100       0        0.00
1                                    100       0        0.00
2                                    100       0        0.00
3                                    100       0        0.00
4                                    100       4        0.04
5                                    100      11        0.11
6                                    100      20        0.20
7                                    100      26        0.26
8                                    100      27        0.27
9                                    100      41        0.41


In [7]:
def ks_statistic(y_true, y_prob):
    """
    Calculates the Kolmogorov-Smirnov statistic.
    """
    df = pd.DataFrame({'y_true': y_true, 'y_prob': y_prob})
    
    # Separate events (1s) and non-events (0s)
    df_event = df[df['y_true'] == 1].sort_values('y_prob', ascending=True)
    df_non_event = df[df['y_true'] == 0].sort_values('y_prob', ascending=True)
    
    # Calculate cumulative distributions
    cdf_event = np.linspace(1/len(df_event), 1, len(df_event))
    cdf_non_event = np.linspace(1/len(df_non_event), 1, len(df_non_event))
    
    # Merge on probability and fill NaNs
    df_merged = pd.merge_asof(df_non_event, df_event, on='y_prob', direction='nearest')
    df_merged = df_merged.fillna(method='ffill').fillna(method='bfill')
    
    # Interpolate to find cumulative probabilities at matching points
    merged_cdf_event = np.interp(df_non_event['y_prob'], df_event['y_prob'], cdf_event)
    
    # Calculate the maximum difference
    max_diff = np.max(np.abs(cdf_non_event - merged_cdf_event))
    
    return max_diff

# Calculate the KS statistic
ks_val = ks_statistic(y, y_prob)
print(f"\nKS Statistic: {ks_val:.4f}")


KS Statistic: 0.5674


C:\Users\Rashmi\AppData\Local\Temp\ipykernel_2856\3894431375.py:17: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_merged = df_merged.fillna(method='ffill').fillna(method='bfill')
